In [1]:
import os
import shutil
import random

train_path = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/train'
test_path = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/test'
filtered_train_path = '/kaggle/working/filtered_train'
filtered_test_path = '/kaggle/working/filtered_test'

classes_to_keep = ['0', '3', '4']

os.makedirs(filtered_train_path, exist_ok=True)
os.makedirs(filtered_test_path, exist_ok=True)

def copy_images(src_dir, dst_dir, class_name,num_class):
    os.makedirs(os.path.join(dst_dir, class_name))
    src_class_dir = os.path.join(src_dir, class_name)
    dst_class_dir = os.path.join(dst_dir, class_name)
    images = os.listdir(src_class_dir)
    random.shuffle(images)
    for img in images[:num_class]:
        shutil.copy(os.path.join(src_class_dir, img), os.path.join(dst_class_dir, img))

for class_name in classes_to_keep:
    copy_images(train_path, filtered_train_path, class_name,2286)

for class_name in classes_to_keep:
    copy_images(test_path, filtered_test_path, class_name,639)

for dir_path in [filtered_train_path, filtered_test_path]:
    print(f"Directory: {dir_path}")
    for class_name in classes_to_keep:
        print(f"{class_name}: {len(os.listdir(os.path.join(dir_path, class_name)))} images")

Directory: /kaggle/working/filtered_train
0: 2286 images
3: 757 images
4: 173 images
Directory: /kaggle/working/filtered_test
0: 639 images
3: 223 images
4: 51 images


# Loading Datasets

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = ImageFolder(root="/kaggle/working/filtered_train", transform=transform)
test_dataset = ImageFolder(root="/kaggle/working/filtered_test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training

In [3]:
import time

def train(model, train_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        start_time = time.time()  # Record start time of epoch
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            batch_size = inputs.size(0)  # Get the batch size
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * batch_size
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += batch_size
        
        epoch_loss = running_loss / total_samples
        epoch_acc = correct_predictions / total_samples
        end_time = time.time()  # Record end time of epoch
        epoch_time = end_time - start_time  # Calculate time taken for epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, Time: {epoch_time:.2f} seconds")


# Performance metrics

In [4]:
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate(model, test_loader):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    return accuracy, precision, recall, f1



# Mobilenet

In [5]:
from torchvision.models import mobilenet_v2
model_mobilenet = mobilenet_v2(pretrained=True)
num_ftrs = model_mobilenet.classifier[1].in_features
model_mobilenet.classifier[1] = nn.Linear(num_ftrs, 3)  # Assuming 3 classes

model_mobilenet = model_mobilenet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mobilenet.parameters(), lr=0.001)
train(model_mobilenet, train_loader, criterion, optimizer, num_epochs=20)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 98.3MB/s]


Epoch 1/20, Loss: 0.3364, Accuracy: 0.8682, Time: 18.85 seconds
Epoch 2/20, Loss: 0.1953, Accuracy: 0.9260, Time: 16.10 seconds
Epoch 3/20, Loss: 0.1240, Accuracy: 0.9562, Time: 16.12 seconds
Epoch 4/20, Loss: 0.0996, Accuracy: 0.9658, Time: 16.22 seconds
Epoch 5/20, Loss: 0.0736, Accuracy: 0.9770, Time: 16.19 seconds
Epoch 6/20, Loss: 0.0499, Accuracy: 0.9820, Time: 16.26 seconds
Epoch 7/20, Loss: 0.0762, Accuracy: 0.9764, Time: 16.32 seconds
Epoch 8/20, Loss: 0.0319, Accuracy: 0.9894, Time: 16.38 seconds
Epoch 9/20, Loss: 0.0675, Accuracy: 0.9779, Time: 16.42 seconds
Epoch 10/20, Loss: 0.0539, Accuracy: 0.9810, Time: 16.46 seconds
Epoch 11/20, Loss: 0.0644, Accuracy: 0.9764, Time: 16.54 seconds
Epoch 12/20, Loss: 0.0516, Accuracy: 0.9832, Time: 16.61 seconds
Epoch 13/20, Loss: 0.0268, Accuracy: 0.9928, Time: 16.73 seconds
Epoch 14/20, Loss: 0.1566, Accuracy: 0.9475, Time: 16.62 seconds
Epoch 15/20, Loss: 0.0275, Accuracy: 0.9925, Time: 16.73 seconds
Epoch 16/20, Loss: 0.0283, Accurac

In [6]:
accuracy, precision, recall, f1 = evaluate(model_mobilenet, test_loader)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")



Accuracy: 0.9441
Precision: 0.9462
Recall: 0.9441
F1-score: 0.9449


# Densenet

In [7]:
from torchvision.models import densenet121

# Load pre-trained DenseNet-121 model
model_densenet = densenet121(pretrained=True)

# Modify the classifier to match the number of classes in your dataset
num_ftrs = model_densenet.classifier.in_features
model_densenet.classifier = nn.Linear(num_ftrs, 3)  # Assuming 3 classes

# Move the model to the device (GPU if available)
model_densenet = model_densenet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_densenet.parameters(), lr=0.001)

# Train the model
train(model_densenet, train_loader, criterion, optimizer, num_epochs=10)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 124MB/s] 


Epoch 1/10, Loss: 0.3838, Accuracy: 0.8495, Time: 37.09 seconds
Epoch 2/10, Loss: 0.2121, Accuracy: 0.9251, Time: 36.76 seconds
Epoch 3/10, Loss: 0.1626, Accuracy: 0.9437, Time: 36.75 seconds
Epoch 4/10, Loss: 0.1632, Accuracy: 0.9400, Time: 36.71 seconds
Epoch 5/10, Loss: 0.1164, Accuracy: 0.9602, Time: 36.74 seconds
Epoch 6/10, Loss: 0.1167, Accuracy: 0.9580, Time: 36.72 seconds
Epoch 7/10, Loss: 0.0980, Accuracy: 0.9633, Time: 36.82 seconds
Epoch 8/10, Loss: 0.0853, Accuracy: 0.9695, Time: 36.70 seconds
Epoch 9/10, Loss: 0.0934, Accuracy: 0.9667, Time: 36.77 seconds
Epoch 10/10, Loss: 0.0540, Accuracy: 0.9792, Time: 36.83 seconds


In [8]:
accuracy, precision, recall, f1 = evaluate(model_densenet, test_loader)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")



Accuracy: 0.9255
Precision: 0.9312
Recall: 0.9255
F1-score: 0.9273


# Resnet

In [9]:
from torchvision.models import resnet18

# Load pre-trained ResNet-18 model
model_resnet18 = resnet18(pretrained=True)

# Modify the classifier to match the number of classes in your dataset
num_ftrs = model_resnet18.fc.in_features
model_resnet18.fc = nn.Linear(num_ftrs, 3)  # Assuming 3 classes

# Move the model to the device (GPU if available)
model_resnet18 = model_resnet18.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.001)

# Train the model
train(model_resnet18, train_loader, criterion, optimizer, num_epochs=10)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s] 


Epoch 1/10, Loss: 0.4554, Accuracy: 0.8218, Time: 14.66 seconds
Epoch 2/10, Loss: 0.2451, Accuracy: 0.9111, Time: 14.55 seconds
Epoch 3/10, Loss: 0.1728, Accuracy: 0.9369, Time: 14.40 seconds
Epoch 4/10, Loss: 0.1278, Accuracy: 0.9546, Time: 14.49 seconds
Epoch 5/10, Loss: 0.0998, Accuracy: 0.9639, Time: 14.31 seconds
Epoch 6/10, Loss: 0.0974, Accuracy: 0.9639, Time: 14.32 seconds
Epoch 7/10, Loss: 0.0822, Accuracy: 0.9689, Time: 14.45 seconds
Epoch 8/10, Loss: 0.0579, Accuracy: 0.9801, Time: 14.32 seconds
Epoch 9/10, Loss: 0.0507, Accuracy: 0.9813, Time: 14.39 seconds
Epoch 10/10, Loss: 0.0701, Accuracy: 0.9748, Time: 14.39 seconds


In [10]:
accuracy, precision, recall, f1 = evaluate(model_resnet18, test_loader)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Accuracy: 0.8828
Precision: 0.9060
Recall: 0.8828
F1-score: 0.8890


# Ensemble Model

In [11]:
class EnsembleModel(nn.Module):
    def __init__(self, resnet_model, mobilenet_model, densenet_model):
        super(EnsembleModel, self).__init__()
        self.resnet_model = resnet_model
        self.mobilenet_model = mobilenet_model
        self.densenet_model = densenet_model
    
    def forward(self, x):
        resnet_output = self.resnet_model(x)
        mobilenet_output = self.mobilenet_model(x)
        densenet_output = self.densenet_model(x)
        ensemble_output = (resnet_output + mobilenet_output + densenet_output) / 3.0  # Averaging predictions
        return ensemble_output

ensemble_model = EnsembleModel(model_resnet18, model_mobilenet, model_densenet)
ensemble_model = ensemble_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.001)
train(ensemble_model, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.0536, Accuracy: 0.9807, Time: 55.47 seconds
Epoch 2/10, Loss: 0.0412, Accuracy: 0.9869, Time: 55.82 seconds
Epoch 3/10, Loss: 0.0346, Accuracy: 0.9876, Time: 55.98 seconds
Epoch 4/10, Loss: 0.0369, Accuracy: 0.9869, Time: 56.13 seconds
Epoch 5/10, Loss: 0.0454, Accuracy: 0.9854, Time: 56.06 seconds
Epoch 6/10, Loss: 0.0128, Accuracy: 0.9966, Time: 56.11 seconds
Epoch 7/10, Loss: 0.0312, Accuracy: 0.9904, Time: 56.09 seconds
Epoch 8/10, Loss: 0.0340, Accuracy: 0.9885, Time: 56.08 seconds
Epoch 9/10, Loss: 0.0585, Accuracy: 0.9807, Time: 56.05 seconds
Epoch 10/10, Loss: 0.0309, Accuracy: 0.9882, Time: 55.99 seconds


In [12]:
accuracy, precision, recall, f1 = evaluate(ensemble_model, test_loader)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Accuracy: 0.9507
Precision: 0.9503
Recall: 0.9507
F1-score: 0.9498


In [ ]:
till here

In [15]:
class Ensemble(nn.Module):
    def __init__(self, resnet_model, mobilenet_model, densenet_model):
        super(Ensemble, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.densenet = densenet121(pretrained=True)
        self.mobilenet = mobilenet_v2(pretrained=True)
        self.fc = nn.Linear(1000*3, 3)  # Output layer with 3 classes
        
    def forward(self, x):
        features_resnet = self.resnet(x)
        features_densenet = self.densenet(x)
        features_mobilenet = self.mobilenet(x)
        features_concat = torch.cat((features_resnet, features_densenet, features_mobilenet), dim=1)
        output = self.fc(features_concat)
        return output

# Create an instance of the ensemble model
ensemble = Ensemble(model_resnet18, model_mobilenet, model_densenet)
ensemble = ensemble.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ensemble.parameters(), lr=0.001)
train(ensemble, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.8129, Accuracy: 0.7833, Time: 55.38 seconds
Epoch 2/10, Loss: 0.2813, Accuracy: 0.8905, Time: 55.74 seconds
Epoch 3/10, Loss: 0.1950, Accuracy: 0.9235, Time: 55.96 seconds
Epoch 4/10, Loss: 0.1300, Accuracy: 0.9493, Time: 56.18 seconds
Epoch 5/10, Loss: 0.1213, Accuracy: 0.9552, Time: 56.23 seconds
Epoch 6/10, Loss: 0.1268, Accuracy: 0.9558, Time: 56.27 seconds
Epoch 7/10, Loss: 0.0968, Accuracy: 0.9639, Time: 56.24 seconds
Epoch 8/10, Loss: 0.0726, Accuracy: 0.9742, Time: 56.22 seconds
Epoch 9/10, Loss: 0.0762, Accuracy: 0.9754, Time: 56.28 seconds
Epoch 10/10, Loss: 0.0943, Accuracy: 0.9680, Time: 56.32 seconds


In [17]:
accuracy, precision, recall, f1 = evaluate(ensemble, test_loader)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Accuracy: 0.9409
Precision: 0.9406
Recall: 0.9409
F1-score: 0.9387


In [19]:
torch.save(ensemble_model,'ensemble_model.pth')

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Load the image and apply transformations
image_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/val/4/9070207R.png"
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Ensure that the image has three channels (RGB)
if image.mode != 'RGB':
    image = image.convert('RGB')

input_image = transform(image).unsqueeze(0)  # Add batch dimension

# Normalize the image
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_image = normalize(input_image)

model=model_resnet

# Pass the image through the model
with torch.no_grad():
    model.eval()
    output = model(input_image.to(device))

# Get predicted class
_, predicted_class = torch.max(output, 1)
predicted_class_index = predicted_class.item()

# Interpret prediction
class_names = ["Healthy", "Moderate", "Severe"]  # Replace with your actual class names
predicted_class_name = class_names[predicted_class_index]

print(f"The model predicts the image to belong to: {predicted_class_name}")

In [3]:

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.densenet = densenet121(pretrained=True)
        self.resnet = resnet50(pretrained=True)
        self.alexnet = alexnet(pretrained=True)
        self.fc = nn.Linear(3000, 3)  # Output layer with 3 classes
        
    def forward(self, x):
        features_densenet = self.densenet(x)
        features_resnet = self.resnet(x)
        features_alexnet = self.alexnet(x)
        features_concat = torch.cat((features_densenet, features_resnet, features_alexnet), dim=1)
        output = self.fc(features_concat)
        return output
    
model = CustomModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 106MB/s] 
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may

In [4]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/10], Loss: 1.4619
Epoch [2/10], Loss: 0.6146
Epoch [3/10], Loss: 0.3748
Epoch [4/10], Loss: 0.2896
Epoch [5/10], Loss: 0.3177
Epoch [6/10], Loss: 0.1920
Epoch [7/10], Loss: 0.1214
Epoch [8/10], Loss: 0.1286
Epoch [9/10], Loss: 0.1360
Epoch [10/10], Loss: 0.1168


In [5]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {100 * accuracy:.2f}%")

Accuracy on test set: 85.36%


In [6]:
import torch
from torchvision import transforms
from PIL import Image

# Load the image and apply transformations
image_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/0/9004184_1.png"
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Ensure that the image has three channels (RGB)
if image.mode != 'RGB':
    image = image.convert('RGB')

input_image = transform(image).unsqueeze(0)  # Add batch dimension

# Normalize the image
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_image = normalize(input_image)

# Pass the image through the model
with torch.no_grad():
    model.eval()
    output = model(input_image.to(device))

# Get predicted class
_, predicted_class = torch.max(output, 1)
predicted_class_index = predicted_class.item()

# Interpret prediction
class_names = ["Healthy", "Moderate", "Severe"]  # Replace with your actual class names
predicted_class_name = class_names[predicted_class_index]
x=predicted_class_index

print(f"The model predicts the image to belong to: {predicted_class_name,x}")

The model predicts the image to belong to: Healthy


In [1]:
train_path = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/train'
test_path = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/test'

In [9]:
import os
os.listdir('/kaggle/input/knee-osteoarthritis-dataset-with-severity/train')

['2', '0', '3', '1', '4']

In [8]:
# Save the model's state_dict
torch.save(model.state_dict(), "ensemble_model.pth")


NameError: name 'torch' is not defined

In [ ]:
import torch
from torchvision.transforms import transforms
from PIL import Image
import numpy as np

# Load the model
model = CustomModel()
model.load_state_dict(torch.load("model.pth"))
model.eval()

# Define the transformation for the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize for single-channel images
])

# Load and preprocess the image
image_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/val/3/9023348L.png"  # Change this to the path of your image
image = Image.open(image_path)
image_tensor = transform(image).unsqueeze(0)

# Perform inference
with torch.no_grad():
    output = model(image_tensor)

# Get the predicted class
predicted_class = torch.argmax(output).item()

# Define the class labels
class_labels = ["Healthy", "Moderate", "Severe"]

# Print the predicted class label
print("Predicted class:", class_labels[predicted_class])
